In [3]:
import yaml
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [4]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/sisyphus/AthenaInsights/notebooks/data_cleaning'

In [5]:
client = boto3.client('s3')

In [6]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [7]:
paginator = client.get_paginator('list_objects_v2')

# List objects under the specified prefix
page_iterator = paginator.paginate(Bucket=bucket, Prefix='AthenaInsights/latest_data/', Delimiter='/')

for page in page_iterator:
    if 'CommonPrefixes' in page:
        for folder in page['CommonPrefixes']:
            print(folder['Prefix'])

AthenaInsights/latest_data/csv/
AthenaInsights/latest_data/data_prep/
AthenaInsights/latest_data/dependent_variable/
AthenaInsights/latest_data/feature_prep/
AthenaInsights/latest_data/model/
AthenaInsights/latest_data/parquet/
AthenaInsights/latest_data/reduced_autocorelation/


### dependent variable

In [8]:
client = boto3.client('s3')

In [9]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [10]:
# dependent_variable_loc = 's3://sisyphus-general-bucket/AthenaInsights/latest_data/dependent_variable/stock_bars_1min.parquet'
# dependent_variable = pd.read_parquet(dependent_variable_loc, columns=['category', 'close_sma_5m' ])

### all paths

In [11]:
paginator = client.get_paginator('list_objects_v2')

# List objects under the specified prefix
page_iterator = paginator.paginate(Bucket=bucket, Prefix='AthenaInsights/latest_data/', Delimiter='/')

for page in page_iterator:
    if 'CommonPrefixes' in page:
        for folder in page['CommonPrefixes']:
            print(folder['Prefix'])

AthenaInsights/latest_data/csv/
AthenaInsights/latest_data/data_prep/
AthenaInsights/latest_data/dependent_variable/
AthenaInsights/latest_data/feature_prep/
AthenaInsights/latest_data/model/
AthenaInsights/latest_data/parquet/
AthenaInsights/latest_data/reduced_autocorelation/


In [12]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

### independent variables

In [13]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/latest_data/feature_prep/')

In [14]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if 'base' in k and '1min' in k: #('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_macd.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_rsi.parquet']

In [15]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if 'base' in k and '2min' in k: #('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base_macd.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base_rsi.parquet']

In [16]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [17]:
len(paths)

151

In [18]:
import pyarrow.dataset as ds
from tqdm import tqdm

In [19]:
# names = []
# time_periods = []
# from pyarrow.parquet import read_metadata
# x = read_metadata(path)
# dir(x)
# x.schema[0].path

In [26]:
if not os.path.exists('all_info_independent_vars.csv'):
    all_info_independent_vars = pd.DataFrame(columns=['name', 'time_period', 'path', 'cols'])

    for path in tqdm(paths):
        name = path.split('/')[-1].replace('.parquet', '')
        time_period = name.replace('stock_bars_', '').split('_')[0]
        df = ds.dataset(path).scanner().head(1).to_pandas()
        columns = list(df.columns)
        all_info_independent_vars.loc[all_info_independent_vars.shape[0]] = [name, time_period, path, columns]
    all_info_independent_vars.to_csv('all_info_independent_vars.csv', index=False)
else:
    print('readinfg from local all_info_independent_vars.csv')
    all_info_independent_vars = pd.read_csv('all_info_independent_vars.csv')

100%|██████████| 151/151 [01:28<00:00,  1.71it/s]


In [27]:
all_info_independent_vars.head(10)

,name,time_period,path,cols
0,stock_bars_100D_base,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[open, high, low, close, volume, symbol, time_..."
1,stock_bars_100D_base_macd,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[symbol, close_ema_5m, close_ema_8m, close_ema..."
2,stock_bars_100D_base_rsi,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[close, close_ema_5m, close_sma_5m, close_ema_..."
3,stock_bars_100D_diff,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[open_diff, high_diff, low_diff, close_diff, v..."
4,stock_bars_100D_diff_macd,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[symbol, close_diff_ema_5m, close_diff_ema_8m,..."
5,stock_bars_100D_diff_rsi,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[close_diff, close_diff_ema_5m, close_diff_sma..."
6,stock_bars_10D_base,10D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[open, high, low, close, volume, symbol, time_..."
7,stock_bars_10D_base_macd,10D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[symbol, close_ema_2m, close_ema_3m, close_ema..."
8,stock_bars_10D_base_rsi,10D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[close, close_ema_5m, close_sma_5m, close_ema_..."
9,stock_bars_10D_diff,10D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[open_diff, high_diff, low_diff, close_diff, v..."


In [28]:
all_info_independent_vars.shape

(151, 4)

In [29]:
all_info_independent_vars[all_info_independent_vars.path.str.contains('latest_data/feature_prep/stock_bars_1min_base_other_features.parquet')].cols.values

array([list(['open', 'high', 'low', 'close', 'volume', 'open_ema_2m', 'open_sma_2m', 'high_ema_2m', 'high_sma_2m', 'low_ema_2m', 'low_sma_2m', 'close_ema_2m', 'close_sma_2m', 'volume_ema_2m', 'volume_sma_2m', 'open_ema_3m', 'open_sma_3m', 'high_ema_3m', 'high_sma_3m', 'low_ema_3m', 'low_sma_3m', 'close_ema_3m', 'close_sma_3m', 'volume_ema_3m', 'volume_sma_3m', 'open_ema_5m', 'open_sma_5m', 'high_ema_5m', 'high_sma_5m', 'low_ema_5m', 'low_sma_5m', 'close_ema_5m', 'close_sma_5m', 'volume_ema_5m', 'volume_sma_5m', 'open_ema_8m', 'open_sma_8m', 'high_ema_8m', 'high_sma_8m', 'low_ema_8m', 'low_sma_8m', 'close_ema_8m', 'close_sma_8m', 'volume_ema_8m', 'volume_sma_8m', 'open_ema_12m', 'open_sma_12m', 'high_ema_12m', 'high_sma_12m', 'low_ema_12m', 'low_sma_12m', 'close_ema_12m', 'close_sma_12m', 'volume_ema_12m', 'volume_sma_12m', 'open_ema_10m', 'open_sma_10m', 'high_ema_10m', 'high_sma_10m', 'low_ema_10m', 'low_sma_10m', 'close_ema_10m', 'close_sma_10m', 'volume_ema_10m', 'volume_sma_10m', '

In [30]:
paths_config_independent_vars = dict().fromkeys(list(all_info_independent_vars.time_period.unique()),)
for row in all_info_independent_vars.iterrows():
    if paths_config_independent_vars[row[1]['time_period']] is None:
        paths_config_independent_vars[row[1]['time_period']] = {}
    # print(row[1]['time_period'], row[1]['name'] )
    paths_config_independent_vars[row[1]['time_period']][row[1]['name']] = {}
    paths_config_independent_vars[row[1]['time_period']][row[1]['name']]['path'] = row[1]['path']
    paths_config_independent_vars[row[1]['time_period']][row[1]['name']]['cols'] = row[1]['cols']

In [31]:
paths_config_independent_vars.keys()
# paths_config_independent_vars['100D'].keys()
paths_config_independent_vars['100D'].keys()
# paths_config_independent_vars['100D']['stock_bars_100D_base'].keys()

dict_keys(['stock_bars_100D_base', 'stock_bars_100D_base_macd', 'stock_bars_100D_base_rsi', 'stock_bars_100D_diff', 'stock_bars_100D_diff_macd', 'stock_bars_100D_diff_rsi'])

### dependent variables

In [32]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/latest_data/dependent_variable/')

In [33]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [34]:
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/dependent_variable/stock_bars_1min.parquet']

In [35]:
if not os.path.exists('all_info_dependent_vars.csv'):
    all_info_dependent_vars = pd.DataFrame(columns=['name', 'time_period', 'path', 'cols'])

    for path in tqdm(paths):
        name = path.split('/')[-1].replace('.parquet', '')
        time_period = name.replace('stock_bars_', '').split('_')[0]
        df = ds.dataset(path).scanner().head(1).to_pandas()
        columns = list(df.columns)
        all_info_dependent_vars.loc[all_info_dependent_vars.shape[0]] = [name, time_period, path, columns]
    all_info_dependent_vars.to_csv('all_info_dependent_vars.csv', index=False)
else:
    print('readinfg from local all_info_dependent_vars.csv')
    all_info_dependent_vars = pd.read_csv('all_info_dependent_vars.csv')

readinfg from local all_info_dependent_vars.csv


In [36]:
all_info_dependent_vars

,name,time_period,path,cols
0,stock_bars_1min,1min,s3://sisyphus-general-bucket/AthenaInsights/la...,"['open', 'high', 'low', 'close', 'close_sma_5m..."


In [37]:
paths_config_dependent_vars = dict().fromkeys(list(all_info_dependent_vars.time_period.unique()),)
for row in all_info_dependent_vars.iterrows():
    if paths_config_dependent_vars[row[1]['time_period']] is None:
        paths_config_dependent_vars[row[1]['time_period']] = {}
    # print(row[1]['time_period'], row[1]['name'] )
    paths_config_dependent_vars[row[1]['time_period']][row[1]['name']] = {}
    paths_config_dependent_vars[row[1]['time_period']][row[1]['name']]['path'] = row[1]['path']
    paths_config_dependent_vars[row[1]['time_period']][row[1]['name']]['cols'] = row[1]['cols']

In [38]:
paths_config_dependent_vars

{'1min': {'stock_bars_1min': {'path': 's3://sisyphus-general-bucket/AthenaInsights/latest_data/dependent_variable/stock_bars_1min.parquet',
   'cols': "['open', 'high', 'low', 'close', 'close_sma_5m', 'category', 'future_highs', 'future_lows', 'slopes']"}}}

### final config file

In [39]:
all_paths_config = {}
all_paths_config['feature_store'] = paths_config_independent_vars
all_paths_config['dependent_variable'] = paths_config_dependent_vars

In [40]:
with open('../../config/spy_30min_v1/features.yaml', 'w+') as ff:
    yaml.dump(all_paths_config, ff)

In [4]:
x = ['open_lag_1', 'high_lag_1', 'low_lag_1', 'close_lag_1', 'open_lag_2',
       'high_lag_2', 'low_lag_2', 'close_lag_2', 'open_lag_3', 'high_lag_3',
       'low_lag_3', 'close_lag_3', 'open_lag_4', 'high_lag_4', 'low_lag_4',
       'close_lag_4', 'open_lag_5', 'high_lag_5', 'low_lag_5', 'close_lag_5',
       'open_lag_6', 'high_lag_6', 'low_lag_6', 'close_lag_6', 'open_lag_7',
       'high_lag_7', 'low_lag_7', 'close_lag_7', 'open_lag_8', 'high_lag_8',
       'low_lag_8', 'close_lag_8', 'open_lag_9', 'high_lag_9', 'low_lag_9',
       'close_lag_9', 'open_lag_10', 'high_lag_10', 'low_lag_10',
       'close_lag_10', 'open_lag_11', 'high_lag_11', 'low_lag_11',
       'close_lag_11', 'open_lag_12', 'high_lag_12', 'low_lag_12',
       'close_lag_12', 'open_lag_13', 'high_lag_13', 'low_lag_13',
       'close_lag_13', 'open_lag_14', 'high_lag_14', 'low_lag_14',
       'close_lag_14', 'open_lag_15', 'high_lag_15', 'low_lag_15',
       'close_lag_15', 'price_change', 'direction', 'cumulative_ups',
       'cumulative_downs', 'direction_change_up_to_down',
       'direction_change_down_to_up', 'prev_cumulative_ups',
       'prev_cumulative_downs', 'hour', 'minute', 'day_of_year', 'local_max_5',
       'local_min_5', 'cumulative_local_max_5', 'cumulative_local_min_5',
       'time_since_prev_max_5', 'time_since_prev_min_5', 'local_max_10',
       'local_min_10', 'cumulative_local_max_10', 'cumulative_local_min_10',
       'time_since_prev_max_10', 'time_since_prev_min_10', 'local_max_15',
       'local_min_15', 'cumulative_local_max_15', 'cumulative_local_min_15',
       'time_since_prev_max_15', 'time_since_prev_min_15', 'local_max_30',
       'local_min_30', 'cumulative_local_max_30', 'cumulative_local_min_30',
       'time_since_prev_max_30', 'time_since_prev_min_30', 'max_today',
       'min_today', 'max_today_session', 'min_today_session']
len(x)

99